[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rsinghlab/pyaging/blob/main/tutorials/tutorial_dnam.ipynb) [![Open In nbviewer](https://img.shields.io/badge/View%20in-nbviewer-orange)](https://nbviewer.jupyter.org/github/rsinghlab/pyaging/blob/main/tutorials/tutorial_dnam.ipynb)

# RRBS DNA methylation

This tutorial focuses on predicting age from Mus musculus reduced-representation bisulfite sequencing (RRBS) data. There are a few clocks available that were trained on RRBS data. Moreover, it is possible to use Horvath's mammalian clocks by converting the genomic location to the probes in the Horvath methylation array.

In [1]:
import pandas as pd
import pyaging as pya
import os
import numpy as np

## Download and load example data

Let's download the publicly available dataset GSE130735 with RRBS samples from mouse. Given it is RRBS, there are millions of CpG sites.

In [2]:
pya.data.download_example_data('GSE130735')

|-----> 🏗️ Starting download_example_data function
|-----------> Data found in pyaging_data/GSE130735_subset.pkl
|-----> 🎉 Done! [0.5066s]


|-----------> in progress: 1.0022%

|-----------> in progress: 2.0043%

|-----------> in progress: 3.0065%

|-----------> in progress: 4.0086%

...

|-----------> in progress: 97.2094%

|-----------> in progress: 98.2115%

|-----------> in progress: 99.2137%

|-----------> in progress: 100.0000%


|-----> 🎉 Done! [42.4343s]


In [3]:
df = pd.read_pickle('pyaging_data/GSE130735_subset.pkl')

It is important to note that the features for RRBS clocks are the genomic coordinates in the format below.

In [4]:
df.head()

,chr1:3020814,chr1:3020842,chr1:3020877,chr1:3020891,chr1:3020945,chr1:3020971,chr1:3020987,chr1:3021012,chr1:3037802,chr1:3037820,...,chrY:1825397,chrY:4682362,chrY:32122892,chrY:85867071,chrY:85867083,chrY:85867117,chrY:85867137,chrY:85867139,chrY:85867178,chrY:88224179
GSM3752631,0.609,0.25,0.408,0.189,0.068,0.373,0.571,0.252,0.333,0.158,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSM3752625,NaN,NaN,0.973,0.984,0.912,0.915,0.987,0.974,0.991,0.932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSM3752634,NaN,NaN,0.526,0.131,0.000,0.038,0.469,0.769,0.772,0.146,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSM3752620,0.931,0.92,0.988,0.949,0.897,0.921,0.907,0.958,1.000,0.867,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSM3752622,NaN,NaN,0.205,0.382,0.091,0.132,0.174,0.227,0.108,0.053,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Convert data to AnnData object

AnnData objects are highly flexible and are thus our preferred method of organizing data for age prediction.

In [5]:
adata = pya.pp.df_to_adata(df, imputer_strategy='mean') # knn might be a bit slow

|-----> 🏗️ Starting df_to_adata function
|-----> ⚙️ Create anndata object started
|-----> ✅ Create anndata object finished [3.8270s]
|-----> ⚙️ Add metadata to anndata started
|-----------? No metadata provided. Leaving adata.obs empty
|-----> ⚠️ Add metadata to anndata finished [0.0012s]
|-----> ⚙️ Log data statistics started
|-----------> There are 14 observations
|-----------> There are 1778324 features
|-----------> Total missing values: 6322346
|-----------> Percentage of missing values: 25.39%
|-----> ✅ Log data statistics finished [0.0586s]
|-----> ⚙️ Impute missing values started
|-----------> Imputing missing values using mean strategy
|-----> ✅ Impute missing values finished [1.3584s]
|-----> ⚙️ Add imputer strategy to adata.uns started
|-----> ✅ Add imputer strategy to adata.uns finished [0.0006s]
|-----> 🎉 Done! [5.2660s]


|-----> ✅ Create anndata object finished [0.9810s]


|-----> ⚙️ Add metadata to anndata started


|-----------? No metadata provided. Leaving adata.obs empty


|-----> ⚠️ Add metadata to anndata finished [0.0005s]


...

|-----------> Imputing missing values using mean strategy


|-----> ✅ Impute missing values finished [0.3931s]


|-----> ⚙️ Add imputer strategy to adata.uns started


|-----> ✅ Add imputer strategy to adata.uns finished [0.0003s]


|-----> 🎉 Done! [1.3951s]


This is what the `adata` object looks like:

## Predict age with RRBS clocks

We can either predict one clock at once or all at the same time. For convenience, let's simply input all four available mammalian clocks at once. The function is invariant to the capitalization of the clock name.

In [6]:
pya.pred.predict_age(adata, ['Thompson', 'Meer', 'Petkovich', 'Stubbs'])

|-----> 🏗️ Starting predict_age function
|-----> ⚙️ Set PyTorch device started
|-----------> Using device: cpu
|-----> ✅ Set PyTorch device finished [0.0018s]
|-----> 🕒 Processing clock: thompson
|-----------> ⚙️ Load clock started
|-----------------> Data found in pyaging_data/thompson.pt
|-----------> ✅ Load clock finished [0.4952s]
|-----------> ⚙️ Check features in adata started
|-----------------? 1 out of 582 features (0.17%) are missing: ['chr4:91376687'], etc.
|-----------------> Filling missing features entirely with 0
|-----------------> Added prepared input matrix to adata.obsm[X_thompson]
|-----------> ⚠️ Check features in adata finished [0.0566s]
|-----------> ⚙️ Predict ages with model started
|-----------------> There is no preprocessing necessary
|-----------------> There is no postprocessing necessary
|-----------------> in progress: 100.0000%
|-----------> ✅ Predict ages with model finished [0.0039s]
|-----------> ⚙️ Add predicted ages and clock metadata to adata star

|-----------> Using device: cpu


|-----> ✅ Set PyTorch device finished [0.0004s]


|-----> 🕒 Processing clock: thompson


|-----------> ⚙️ Load clock started


...

|-----------------> in progress: 100.0000%


|-----------> ✅ Predict ages with model finished [0.0204s]


|-----------> ⚙️ Add predicted ages and clock metadata to adata started


|-----------> ✅ Add predicted ages and clock metadata to adata finished [0.0005s]


|-----> 🎉 Done! [5.5415s]


All of the age predictions are in unit of months.

In [7]:
adata.obs.head()

,thompson,meer,petkovich,stubbs
GSM3752631,19.634113,7.315183,8.075177,0.957770
GSM3752625,-1.410461,0.028221,2.953822,-0.074265
GSM3752634,61.058783,21.322178,9.640489,1.389193
GSM3752620,-2.663815,1.611947,3.019351,-0.092710
GSM3752622,20.594114,7.592145,7.104766,0.667168


In [8]:
adata.obs.head()

,thompson,meer,petkovich,stubbs
GSM3752631,19.634113,7.315183,8.075177,0.957770
GSM3752625,-1.410461,0.028221,2.953822,-0.074265
GSM3752634,61.058783,21.322178,9.640489,1.389193
GSM3752620,-2.663815,1.611947,3.019351,-0.092710
GSM3752622,20.594114,7.592145,7.104766,0.667168


Having so much information printed can be overwhelming, particularly when running several clocks at once. In such cases, just set verbose to False.

In [9]:
pya.data.download_example_data('GSE130735', verbose=False)
df = pd.read_pickle('pyaging_data/GSE130735_subset.pkl')
adata = pya.preprocess.df_to_adata(df, imputer_strategy='mean', verbose=False)
pya.pred.predict_age(adata, ['Thompson', 'Meer', 'Petkovich', 'Stubbs'], verbose=False)

In [10]:
adata.obs.head()

,thompson,meer,petkovich,stubbs
GSM3752631,19.634113,7.315183,8.075177,0.957770
GSM3752625,-1.410461,0.028221,2.953822,-0.074265
GSM3752634,61.058783,21.322178,9.640489,1.389193
GSM3752620,-2.663815,1.611947,3.019351,-0.092710
GSM3752622,20.594114,7.592145,7.104766,0.667168


After age prediction, the clocks are added to `adata.obs`. Moreover, the percent of missing values for each clock and other metadata are included in `adata.uns`.

In [11]:
adata

AnnData object with n_obs × n_vars = 14 × 1778324
    obs: 'thompson', 'meer', 'petkovich', 'stubbs'
    var: 'percent_na'
    uns: 'imputer_strategy', 'thompson_percent_na', 'thompson_missing_features', 'thompson_metadata', 'meer_percent_na', 'meer_missing_features', 'meer_metadata', 'petkovich_percent_na', 'petkovich_missing_features', 'petkovich_metadata', 'stubbs_percent_na', 'stubbs_missing_features', 'stubbs_metadata'
    layers: 'X_original', 'X_imputed'

## Predict age with mammalian clocks

We can predict age by converting the genomic locations directly into the probes from Horvath's methylation array. 

In [ ]:
os.system('git clone https://github.com/shorvath/MammalianMethylationConsortium.git')

# Let's read the manifest from the mammalian consortium
annotation_df = pd.read_csv('MammalianMethylationConsortium/Annotations, Amin Haghani/Mammals/Mus_musculus.grcm38.100.HorvathMammalMethylChip40.v1.csv', index_col=0)
annotation_df = annotation_df[~annotation_df.seqnames.isna()]
mm_genomic_locations = 'chr' + annotation_df['seqnames'].astype(str) + ':' + annotation_df['CGstart'].astype(int).astype(str)
mm_genomic_locations = mm_genomic_locations.tolist()
mammalian_probes = annotation_df['CGid'].tolist()
mm_loc_to_probe = dict(zip(mm_genomic_locations, mammalian_probes))

# Let's get the previous RRBS dataset and filter only for the genomic locations in the manifest file
df_columns_set = set(df.columns)
mm_loc_to_probe_set = set(mm_loc_to_probe.keys())
common_columns = df_columns_set.intersection(mm_loc_to_probe_set)
df_converted = df[list(common_columns)].copy()

# Then, convert the genomic location to the probe name
df_converted.columns = [mm_loc_to_probe[col] for col in df_converted.columns]

# Let's clean the GitHub
os.system('rm -r MammalianMethylationConsortium')

In [ ]:
df_converted.head()

Now we can finally put the dataframe into pyaging after defining the species as Mus musculus.

In [ ]:
df_converted['Mus musculus'] = 1
adata_mammalian = pya.pp.df_to_adata(df_converted, imputer_strategy='mean')

|-----> ✅ Create anndata object finished [0.0039s]


|-----> ⚙️ Add metadata to anndata started


|-----------? No metadata provided. Leaving adata.obs empty


|-----> ⚠️ Add metadata to anndata finished [0.0004s]


|-----> ⚙️ Log data statistics started


...

|-----------> Imputing missing values using mean strategy


|-----> ✅ Impute missing values finished [0.0029s]


|-----> ⚙️ Add imputer strategy to adata.uns started


|-----> ✅ Add imputer strategy to adata.uns finished [0.0003s]


|-----> 🎉 Done! [0.0108s]


Let's use these five mammalian predictors.

In [ ]:
pya.pred.predict_age(adata_mammalian, ['Mammalian1', 'Mammalian2', 'Mammalian3', "MammalianLifespan", "MammalianFemale"])

|-----------> Using device: cpu


|-----> ✅ Set PyTorch device finished [0.0004s]


|-----> 🕒 Processing clock: mammalian1


|-----------> ⚙️ Load clock started


|-----------------> Data found in pyaging_data/mammalian1.pt


...

|-----------------> in progress: 100.0000%


|-----------> ✅ Predict ages with model finished [0.0009s]


|-----------> ⚙️ Add predicted ages and clock metadata to adata started


|-----------> ✅ Add predicted ages and clock metadata to adata finished [0.0004s]


|-----> 🎉 Done! [2.9784s]


Note that RRBS clocks are in units of months whereas the mammalian clocks are in units of years.

In [ ]:
adata_mammalian.obs

## Get citation

The doi, citation, and some metadata are automatically added to the AnnData object under `adata.uns[CLOCKNAME_metadata]`.

In [ ]:
adata.uns['thompson_metadata']

In [ ]:
adata.uns['meer_metadata']

In [ ]:
adata.uns['petkovich_metadata']

In [ ]:
adata.uns['stubbs_metadata']

In [ ]:
adata_mammalian.uns['mammalian1_metadata']

In [ ]:
adata_mammalian.uns['mammalianlifespan_metadata']